In [3]:
import gzip
import pandas as pd
import matplotlib.pyplot as plt

years = list(range(1470, 2021))
words = []
data_year = []
with gzip.open('C:/Users/bincl/BA-Thesis/Dataset/1_20000_nopos.gz','rt', encoding='utf-8') as input:
    for line in input:
        values = line.strip().split("\t")
        words.append(values[0]) 
        data = [entry.split(",") for entry in values[1:]]
        year = {entry[0]: entry[1] for entry in data}
        data_year.append(year)
        
df = pd.DataFrame(index=words, columns= years)      
for i, word_data in enumerate(data_year):
    word = words[i]
    for year, value in word_data.items():
        df.at[word, int(year)] = value  # Füge den Wert an der entsprechenden Position im DataFrame ein

# Fülle die NaN-Werte im DataFrame mit 0
df.fillna(0, inplace=True)
print(df) 


               1470  1471  1472 1473  1474 1475  1476 1477  1478  1479  ...  \
Empfehlungen      0     0     0    0     0    0     0    0     0     0  ...   
Fachbereich       0     0     0    0     0    0     0    0     0     0  ...   
Besprechungen     0     0     0    0     0    0     0    0     0     0  ...   
Berechtigung      0     0     0    0     0    0     0    0     0     0  ...   
Beiträgen         0     0     0    0     0    0     0    0     0     0  ...   
...             ...   ...   ...  ...   ...  ...   ...  ...   ...   ...  ...   
geistig           0     0     0    0     0    0     0    0     0     0  ...   
einsetzte         0     0     0    0     0    0     0    0     0     0  ...   
fasziniert        0     0     0    0     0    0     0    0     0     0  ...   
fährt             0     0     0    0     0    0     0    0     0     0  ...   
jeweiligen        0     0     0    0     0    2     0    0     0     0  ...   

                 2011    2012    2013    2014    20

In [8]:
import numpy as np
import pickle

numpy_array = df.values

# Das Numpy-Array enthält die Werte aus dem DataFrame
print(numpy_array)

converted_array = np.empty(numpy_array.shape)
for idx, row in enumerate(numpy_array):
    for jdx, val in enumerate(row):
        try:
            converted_array[idx, jdx] = float(val)
        except (ValueError, TypeError):
            converted_array[idx, jdx] = val

np.savetxt('1grams_TEST.tsv', converted_array, delimiter='\t', fmt='%s')


with open('Uf48.bin.dat', 'wb') as file:
    pickle.dump(numpy_array, file)

[[0 0 0 ... '69628' '60539' 0]
 [0 0 0 ... '13833' '10587' 0]
 [0 0 0 ... '15816' '13468' 0]
 ...
 [0 0 0 ... '54448' '52195' 0]
 [0 0 0 ... '186778' '167686' 0]
 [0 0 0 ... '346882' '298751' 0]]


In [ ]:
from multiprocessing import Pool

def process_line(line):
    values = line.strip().split("\t")
    word = values[0]
    data = [entry.split(",") for entry in values[1:]]
    columns = {entry[0]: entry[1] for entry in data}
    return pd.DataFrame(columns, index=[word])

# Erstellen eines leeren DataFrames mit der ersten Spalte als Index
df = pd.DataFrame()

with gzip.open('C:/Users/bincl/BA-Thesis/1_20000_nopos.gz', 'rt', encoding='utf-8') as input_file:
    lines = input_file.readlines()

    # Anzahl der Prozesse (Kerne) festlegen
    num_processes = 4
    with Pool(num_processes) as pool:
        result = pool.map(process_line, lines)
    
    df = pd.concat(result)

# Umbenennung der Spaltennamen
df.columns.name = 'Word'

print(df)